In [42]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from pathlib import Path
import csv
import seaborn as sns
import numpy as np
import hvplot.pandas
from panel.interact import interact

%matplotlib inline

In [43]:
C04_C06_path = Path('/Users/ameerirfan/Desktop/FT-BTC-Activities/FT-BTC_Project_1/C04_C06_df.csv')
C04_C06_df = pd.read_csv(C04_C06_path)

C04_C06_df.head()


,Date,Location,CompBenchmark,CompYoYChange,SFDetachBenchmark,SFDetachYoYChange,ApartBenchmark,ApartYoYChange
0,2015-07-01,Toronto C01,478800.0,2.99,679500.0,-3.51,395600.0,3.32
1,2015-08-01,Toronto C01,486000.0,4.92,702800.0,-0.70,402100.0,5.63
2,2015-09-01,Toronto C01,488200.0,7.12,687600.0,-1.02,405200.0,8.29
3,2015-10-01,Toronto C01,484500.0,5.22,696100.0,1.49,399400.0,5.38
4,2015-11-01,Toronto C01,483000.0,4.22,706700.0,0.00,396200.0,4.39


In [44]:
realestate_to_df = C04_C06_df
realestate_to_df[["year", "month", "day"]] = C04_C06_df['Date'].str.split("-", expand = True)
realestate_to_df

,Date,Location,CompBenchmark,CompYoYChange,SFDetachBenchmark,SFDetachYoYChange,ApartBenchmark,ApartYoYChange,year,month,day
0,2015-07-01,Toronto C01,478800.0,2.99,679500.0,-3.51,395600.0,3.32,2015,07,01
1,2015-08-01,Toronto C01,486000.0,4.92,702800.0,-0.70,402100.0,5.63,2015,08,01
2,2015-09-01,Toronto C01,488200.0,7.12,687600.0,-1.02,405200.0,8.29,2015,09,01
3,2015-10-01,Toronto C01,484500.0,5.22,696100.0,1.49,399400.0,5.38,2015,10,01
4,2015-11-01,Toronto C01,483000.0,4.22,706700.0,0.00,396200.0,4.39,2015,11,01
...,...,...,...,...,...,...,...,...,...,...,...
961,2020-11-01,Toronto C15,864800.0,6.99,1395400.0,2.20,585000.0,7.29,2020,11,01
962,2020-12-01,Toronto C15,868400.0,5.69,1413300.0,4.15,583600.0,4.11,2020,12,01
963,2021-01-01,Toronto C15,881100.0,5.89,1458800.0,9.67,587400.0,2.35,2021,01,01
964,2021-02-01,Toronto C15,904400.0,6.33,1508100.0,12.23,603600.0,1.96,2021,02,01


In [45]:
realestate_to_df.set_index('year', inplace=True)
realestate_to_df.head()

,Date,Location,CompBenchmark,CompYoYChange,SFDetachBenchmark,SFDetachYoYChange,ApartBenchmark,ApartYoYChange,month,day
year,,,,,,,,,,
2015,2015-07-01,Toronto C01,478800.0,2.99,679500.0,-3.51,395600.0,3.32,07,01
2015,2015-08-01,Toronto C01,486000.0,4.92,702800.0,-0.70,402100.0,5.63,08,01
2015,2015-09-01,Toronto C01,488200.0,7.12,687600.0,-1.02,405200.0,8.29,09,01
2015,2015-10-01,Toronto C01,484500.0,5.22,696100.0,1.49,399400.0,5.38,10,01
2015,2015-11-01,Toronto C01,483000.0,4.22,706700.0,0.00,396200.0,4.39,11,01


In [46]:
# Create a numerical aggregation that groups the data by the year and then averages the results

realestate_by_year = realestate_to_df.groupby("year").mean()
realestate_by_year = realestate_by_year[["CompBenchmark", "SFDetachBenchmark", "ApartBenchmark"]]

# Review the DataFrame
realestate_by_year

,CompBenchmark,SFDetachBenchmark,ApartBenchmark
year,,,
2015,8.155262e+05,1.168881e+06,419080.952381
2016,9.009435e+05,1.323651e+06,447705.952381
2017,1.053688e+06,1.593640e+06,560186.904762
2018,1.090692e+06,1.533967e+06,626414.285714
2019,1.131908e+06,1.548780e+06,664375.595238
2020,1.205935e+06,1.641142e+06,711628.571429
2021,1.246031e+06,1.755621e+06,713402.380952


In [66]:
# Plot realestate prices for the Composite and the other two type of residences. 
# Inclued labels for the x- and y-axes, and a title.
realestate_by_year.hvplot.line(yformatter='%.0f',x="year", xlabel="Year", ylabel="Prices ($ CAD)", title="Real Estate Prices, 2015-2021 - Toronto City")

:NdOverlay   [Variable]
   :Curve   [year]   (value)

In [48]:
#Real Estate prices by year and neighborhood

realestate_by_year_neighborhood = realestate_to_df.groupby(['year', 'Location']).mean()
realestate_by_year_neighborhood = realestate_by_year_neighborhood[["CompBenchmark", "SFDetachBenchmark", "ApartBenchmark"]]
realestate_by_year_neighborhood


CompBenchmark  SFDetachBenchmark  ApartBenchmark
year Location                                                     
2015 Toronto C01   4.835167e+05       6.996333e+05   398500.000000
     Toronto C02   9.277167e+05       1.439683e+06   525583.333333
     Toronto C03   1.071733e+06       1.225650e+06   547183.333333
     Toronto C04   1.158583e+06       1.349867e+06   370033.333333
     Toronto C06   7.886833e+05       8.857333e+05   409850.000000
...                         ...                ...             ...
2021 Toronto C11   1.076900e+06       1.615567e+06   513766.666667
     Toronto C12   2.115367e+06       2.407167e+06   964833.333333
     Toronto C13   1.056233e+06       1.517033e+06   606633.333333
     Toronto C14   9.497667e+05       1.607000e+06   715866.666667
     Toronto C15   9.050667e+05       1.506233e+06   603800.000000

[98 rows x 3 columns]

In [49]:
realestate_by_year_neighborhood.reset_index()



,year,Location,CompBenchmark,SFDetachBenchmark,ApartBenchmark
0,2015,Toronto C01,4.835167e+05,6.996333e+05,398500.000000
1,2015,Toronto C02,9.277167e+05,1.439683e+06,525583.333333
2,2015,Toronto C03,1.071733e+06,1.225650e+06,547183.333333
3,2015,Toronto C04,1.158583e+06,1.349867e+06,370033.333333
4,2015,Toronto C06,7.886833e+05,8.857333e+05,409850.000000
...,...,...,...,...,...
93,2021,Toronto C11,1.076900e+06,1.615567e+06,513766.666667
94,2021,Toronto C12,2.115367e+06,2.407167e+06,964833.333333
95,2021,Toronto C13,1.056233e+06,1.517033e+06,606633.333333
96,2021,Toronto C14,9.497667e+05,1.607000e+06,715866.666667


In [58]:
realestate_by_year_neighborhood = realestate_by_year_neighborhood.round(2)
realestate_by_year_neighborhood

CompBenchmark  SFDetachBenchmark  ApartBenchmark
year Location                                                     
2015 Toronto C01       483517.0           699633.0        398500.0
     Toronto C02       927717.0          1439683.0        525583.0
     Toronto C03      1071733.0          1225650.0        547183.0
     Toronto C04      1158583.0          1349867.0        370033.0
     Toronto C06       788683.0           885733.0        409850.0
...                         ...                ...             ...
2021 Toronto C11      1076900.0          1615567.0        513767.0
     Toronto C12      2115367.0          2407167.0        964833.0
     Toronto C13      1056233.0          1517033.0        606633.0
     Toronto C14       949767.0          1607000.0        715867.0
     Toronto C15       905067.0          1506233.0        603800.0

[98 rows x 3 columns]

In [65]:
realestate_by_year_neighborhood.hvplot.line(
    yformatter='%.0f',
    title="Real Estate Prices Per MLS Division, 2015-2021 - Toronto City ",
    xlabel = "Year",
    ylabel = "Value ($)",
    groupby="Location"
)

:DynamicMap   [Location]
   :NdOverlay   [Variable]
      :Curve   [year]   (value)